## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import time
from datetime import timedelta
import math
import os
import scipy.misc
from scipy.stats import itemfreq
from random import sample
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2
from keras.models import Sequential
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten

In [0]:
import PIL.Image
from IPython.display import display
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/Colab Notebooks/Adv_CV/CNN_Project2/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
os.chdir("/content/drive/My Drive/Colab Notebooks/Adv_CV/CNN_Project2/")
filename = '/content/drive/My Drive/Colab Notebooks/Adv_CV/CNN_Project2/train.zip'
with ZipFile(filename) as z:
  z.extractall()

In [0]:
ids = next(os.walk(project_path + "train/"))[2] # list of names all images in the given path
print("No. of images = ", len(ids))

No. of images =  10222


Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
os.chdir("/content/drive/My Drive/Colab Notebooks/Adv_CV/CNN_Project2/")
filename = '/content/drive/My Drive/Colab Notebooks/Adv_CV/CNN_Project2/test.zip'
with ZipFile(filename) as y:
  y.extractall()

In [0]:
ids = next(os.walk(project_path + "test/"))[2] # list of names all images in the given path
print("No. of images = ", len(ids))

No. of images =  10357


Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
#os.chdir("/content/drive/My Drive/Colab Notebooks/Adv_CV/CNN_Project2/")
filename = '/content/drive/My Drive/Colab Notebooks/Adv_CV/CNN_Project2/sample_submission.csv.zip'
with ZipFile(filename) as x:
  x.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
#os.chdir("/content/drive/My Drive/Colab Notebooks/Adv_CV/CNN_Project2/")
filename = '/content/drive/My Drive/Colab Notebooks/Adv_CV/CNN_Project2/labels.csv.zip'
with ZipFile(filename) as xx:
  xx.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
labels_data=pd.read_csv("labels.csv")

### Print the count of each category of Dogs given in the dataset



In [0]:
labels_data.groupby(['breed']).count()

,id
breed,
affenpinscher,80
afghan_hound,116
african_hunting_dog,86
airedale,107
american_staffordshire_terrier,74
...,...
welsh_springer_spaniel,79
west_highland_white_terrier,81
whippet,95


### Get one-hot encodings of labels

In [0]:
labels_encoded = pd.get_dummies(labels_data.breed)

In [0]:
labels_encoded.head()

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,english_foxhound,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
im_width = 128
im_height = 128
border = 5
x_train = []
y_train = []

In [0]:
path = "/content/drive/My Drive/Colab Notebooks/Adv_CV/CNN_Project2/train/"

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
for f, img in tqdm(labels_data.values):
  train_img = cv2.imread('./train/{}.jpg'.format(f),1)
  train_img_resize = cv2.resize(train_img,(im_width,im_height))
  x_train.append(train_img_resize)
  y_train.append(img)

In [0]:
x_train = np.array(x_train).astype("float32") / 255

In [0]:
from tqdm import tqdm_notebook, tnrange ,tqdm
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.transform import resize

In [0]:
y_train_class = pd.get_dummies(y_train)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train_class, test_size=0.3, random_state=1)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
test_img = pd.read_csv('sample_submission.csv',usecols=[0])


In [0]:
test_img.id

0        000621fb3cbb32d8935728e48679680e
1        00102ee9d8eb90812350685311fe5890
2        0012a730dfa437f5f3613fb75efcd4ce
3        001510bc8570bbeee98c8d80c8a95ec1
4        001a5f3114548acdefa3d4da05474c2e
                       ...               
10352    ffeda8623d4eee33c6d1156a2ecbfcf8
10353    fff1ec9e6e413275984966f745a313b0
10354    fff74b59b758bbbf13a5793182a9bbe4
10355    fff7d50d848e8014ac1e9172dc6762a3
10356    fffbff22c1f51e3dc80c4bf04089545b
Name: id, Length: 10357, dtype: object

In [0]:
ids = next(os.walk(project_path + "test/"))[2] # list of names all images in the given path
print("No. of images = ", len(ids))

No. of images =  10357


Run the below code to load the test image files in x_test_feature

In [0]:
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.id): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f),0)
    img_resize = cv2.resize(img,(im_width, im_height)) 
    x_test_feature.append(img_resize)

Normalize the test data and convert it into 4 dimensions

In [0]:
x_test_feature = np.array(x_test_feature).astype("float32") / 255

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
from keras.callbacks import ModelCheckpoint
# Initialize and Build the Model
BATCH_SIZE = 128
EPOCHS = 10

# Define the Type of Model
model1 = Sequential()
# 1st Conv Layer
model1.add(BatchNormalization(input_shape = (128,128,3)))
model1.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (128, 128, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(3, 3)))

# 2nd Conv Layer
model1.add(Convolution2D(32, 3, 3))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(3, 3)))

# Fully Connected Layer
model1.add(Flatten())
model1.add(Dense(256))
model1.add(Activation('relu'))

# Dense Layer
model1.add(Dense(120))
model1.add(Activation('softmax'))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  from ipykernel import kernelapp as app


### Use batch_size = 128 and epochs = 10 and execute the model

In [0]:
model1.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [0]:
model1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
activation_3 (Activation)    (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 42, 42, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 40, 40, 32)        9248      
_________________________________________________________________
activation_4 (Activation)    (None, 40, 40, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)       

In [0]:
model1.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, 
              validation_data=(X_val, y_val))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7155 samples, validate on 3067 samples
Epoch 1/10
7155/7155 [==============================] - 10s 1ms/step - loss: 4.8164 - acc: 0.0102 - val_loss: 4.7846 - val_acc: 0.0111
Epoch 2/10
7155/7155 [==============================] - 3s 399us/step - loss: 4.7739 - acc: 0.0140 - val_loss: 4.7749 - val_acc: 0.0124
Epoch 3/10
7155/7155 [==============================] - 3s 402us/step - loss: 4.7564 - acc: 0.0180 - val_loss: 4.7661 - val_acc: 0.0163
Epoch 4/10
7155/7155 [==============================] - 3s 399us/step - loss: 4.7363 - acc: 0.0196 - val_loss: 4.7536 - val_acc: 0.0160
Epoch 5/10
7155/7155 [==============================] - 3s 397us/step - loss: 4.7092 - acc: 0.0221 - val_loss: 4.7396 - val_acc: 0.0173
Epoch 6/10
7155/7155 [==============================] - 3s 400us/step - loss: 4.6737 - acc: 0.0261 - val_loss: 4.7188 - val_acc: 0.0199
Epoch 7/10
7155/7155 [===================

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array


In [0]:

data = ImageDataGenerator(validation_split=0.2, 
                          rescale=1/255,
                          samplewise_center=True, # set input mean to 0 over the sample
                          samplewise_std_normalization=True,  # divide inputs by std of the sample
                          rotation_range=90,       # randomly rotate images in the range (degrees, 0 to 180)
                          width_shift_range=0.2,   # randomly shift images horizontally (fraction of total width)
                          height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
                          fill_mode='reflect',     # filling the area outside
                          zoom_range=0.4,          # random zoom
                          horizontal_flip=True,    # randomly flip images
                          vertical_flip=True)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
train_generator = data.flow(X_train, y_train, batch_size=128)


In [0]:
val_generator = data.flow(X_val,y_val,batch_size = 128)

In [0]:
output_imggen = model1.fit_generator(train_generator, epochs=10, validation_data=val_generator, verbose = 2, steps_per_epoch=20, validation_steps=80)


Epoch 1/10
 - 47s - loss: 4.7310 - acc: 0.0172 - val_loss: 5.1225 - val_acc: 0.0190
Epoch 2/10
 - 42s - loss: 4.6920 - acc: 0.0176 - val_loss: 4.8396 - val_acc: 0.0188
Epoch 3/10
 - 43s - loss: 4.6964 - acc: 0.0224 - val_loss: 4.7698 - val_acc: 0.0191
Epoch 4/10
 - 43s - loss: 4.6641 - acc: 0.0234 - val_loss: 4.7336 - val_acc: 0.0204
Epoch 5/10
 - 43s - loss: 4.6553 - acc: 0.0258 - val_loss: 4.6941 - val_acc: 0.0213
Epoch 6/10
 - 43s - loss: 4.6634 - acc: 0.0214 - val_loss: 4.6897 - val_acc: 0.0235
Epoch 7/10
 - 43s - loss: 4.6614 - acc: 0.0187 - val_loss: 4.6932 - val_acc: 0.0199
Epoch 8/10
 - 41s - loss: 4.6439 - acc: 0.0187 - val_loss: 4.6720 - val_acc: 0.0196
Epoch 9/10
 - 46s - loss: 4.6394 - acc: 0.0211 - val_loss: 4.6823 - val_acc: 0.0239
Epoch 10/10
 - 47s - loss: 4.6239 - acc: 0.0246 - val_loss: 4.6587 - val_acc: 0.0194


### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [0]:
output_imggen = model1.fit_generator(train_generator, epochs=10, validation_data=val_generator, verbose = 2, steps_per_epoch=20, validation_steps=80)


Epoch 1/10
 - 48s - loss: 4.6487 - acc: 0.0242 - val_loss: 4.6690 - val_acc: 0.0190
Epoch 2/10
 - 42s - loss: 4.6192 - acc: 0.0305 - val_loss: 4.6754 - val_acc: 0.0215
Epoch 3/10
 - 42s - loss: 4.6071 - acc: 0.0238 - val_loss: 4.6544 - val_acc: 0.0182
Epoch 4/10
 - 42s - loss: 4.6248 - acc: 0.0184 - val_loss: 4.6521 - val_acc: 0.0212
Epoch 5/10
 - 41s - loss: 4.6116 - acc: 0.0234 - val_loss: 4.6399 - val_acc: 0.0252
Epoch 6/10
 - 41s - loss: 4.6094 - acc: 0.0266 - val_loss: 4.6390 - val_acc: 0.0280
Epoch 7/10
 - 41s - loss: 4.5960 - acc: 0.0344 - val_loss: 4.6348 - val_acc: 0.0229
Epoch 8/10
 - 40s - loss: 4.6050 - acc: 0.0232 - val_loss: 4.6339 - val_acc: 0.0243
Epoch 9/10
 - 43s - loss: 4.5930 - acc: 0.0246 - val_loss: 4.6228 - val_acc: 0.0228
Epoch 10/10
 - 41s - loss: 4.5813 - acc: 0.0281 - val_loss: 4.6192 - val_acc: 0.0256


# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [0]:
#import tf.keras.optimizers.SGD as sgd
x = base_model.output

In [0]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
x = Flatten()(x)

In [0]:
# Stacking a new simple convolutional network on top of it    
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)

x = Dense(256, activation='relu')(x)

x = Dense(120, activation='softmax')(x)

# Creating new model. Please note that this is NOT a Sequential() model.
from keras.models import Model
custom_model = Model(input=base_model.input, output=x)

# Make sure that the pre-trained bottom layers are not trainable
for layer in custom_model.layers[:7]:
    layer.trainable = False
#sgd  = SGD(lr = 0.01)
# Do not forget to compile it
custom_model.compile(loss='categorical_crossentropy',
                     optimizer='sgd',
                     metrics=['accuracy'])